In [44]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


from keys import Trip_advisor_key
from keys import geoapify_key

In [34]:
#find the restaurants in the area of intrest

base_url = 'https://api.geoapify.com/v2/places'


#Lat and Lng will host the Latitude And Longitude of the 'districts' of intrest
Districts_df = pd.DataFrame({ 'Location' : ['Toronto Distillery District', 
                                     'Toronto GTA Area',
                                     'Toronto South West',
                                     'Toronto North West',
                                     'Toronto Yorkville',
                                     'Ottawa City Centre'
                              ],
                'Lat' : [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236 ],
                'Lng' : [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009 ],
               })

Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Restaurant Address', 'Lat', 'Lng'])



#preparing the script for the first 'district' choosen

radius = 1000

params = {
    'categories':'catering.restaurant',
    'apiKey': geoapify_key,
    #'limit':20

}




for index, row in Districts_df.iterrows():
    
    latitude = int(row['Lat'])
    longitude = int(row['Lng'])
    

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'


    Restaurant = requests.get(base_url, params = params)

    Restaurant = Restaurant.json()

    for restaurant in restaurant_data.get('features', []):
            
        try:
            
            Restaurants_df.append({'Location' : row['Location'],
                                   'Restaurant Name' : Restaurant  ["properties"]["address_line1"],
                                   'Restaurant Address' : Restaurant  ["properties"]["address_line2"],
                                   'Lat': Restaurant["geometry"]["coordinates"][1],
                                   'Lng': Restaurant["geometry"]["coordinates"][0]
                                  }, ignore_index=True)
        
        except (KeyError, IndexError):
            
            print('Restaurant not found!')


#EXPLORATION TOOLS


#pprint(Restaurant)

#res = len(Restaurant['features'])
#res

Restaurants_df.head()

,Location,Restaurant Name,Restaurant Address,Lat,Lng


In [53]:

# Definizione dei distretti e delle coordinate
Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto South West', 'Toronto North West', 'Toronto Yorkville', 'Ottawa City Centre'],
    'Lat': [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236 ], 
    'Lng': [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009]  
})


# Definizione del dataframe dei ristoranti
Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Restaurant Address', 'Lat', 'Lng'])

# Preparazione dello script per il primo distretto scelto
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    
    for restaurant in response.get('features', []):
            try:
                Restaurants_df = Restaurants_df.append({
                    'Location': row['Location'],
                    'Restaurant Name': restaurant["properties"]["address_line1"],
                    'Restaurant Address': restaurant["properties"]["address_line2"],
                    'Lat': restaurant["geometry"]["coordinates"][1],
                    'Lng': restaurant["geometry"]["coordinates"][0]
                }, ignore_index=True)
            except(KeyError, IndexError):
                print('Errore nella gestione dei dati del ristorante.')
            


# Ora Restaurants_df contiene i dati dei ristoranti nelle vicinanze dei distretti specificati

AttributeError: 'DataFrame' object has no attribute 'append'

,Location,Restaurant Name,Restaurant Address,Lat,Lng
